# **8-2. Weight initialization**
------
학습이 잘 안 되는 이유들 중 하나만 들어볼까요?  
Geoffrey Hinton씨에게 여쭤봤습니다.  
- 우리가 weight를 멍청한 방식으로 초기화 한다     

Weight 초기화를 잘 해야 좋은 결과가 나오는데, 지금까지는 0으로 초기화 하거나, 임의의 값을 넣는 방식으로 초기화를 해오고 있었다!   
0으로 초기화 하는 방식은 굉장히 안 좋은 방식인데, 역전파 단계에서 W를 곱할 때 W가 0이면 gradient가 0이 되기 때문!   

그럼 RBM으로 weight 초기화 하는 건 어떤데

## **RBM(Restricted Boltsmann Machine)**
- Restricted: 레이어 안에서는 연결이 없다
- 다른 레이어들 끼리는 Fully-connected
- 각 층에서 계산된 Weight를 인코딩(forward)/ 디코딩(backward)

### **Deep Belief Network - RBM을 가중치 초기화에 적용**
Pre - training 단계에서 RBM 적용   
stack 쌓는다고 생각하면 편함!   

1. 일단 무작위로 Weight를 초기화 시킨다.
2. 2개의 layer를 RBM으로 학습한다.
3. 2개의 layer위에 한 층을 더 쌓고, 2번째 layer와 새로 쌓은 3번째 layer끼리 RBM으로 학습을 진행한다. 1번째 층과 2번째 층에서의 학습으로 나온 W값은 고정된다.
4. 3번을 반복한다.
5. 구해진 W를 가지고 전체 층에 역전파 시킨다.

그래서 RBM 지금도 많이 쓰나요?    
아니요. RBM 너무 복잡해요...    
RBM보다 더 간단하게 Weight를 초기화 할 수 있는 방법 없을까?   

## **Xavier(2010) / He(2015) initialization**
수식을 이용한 초기화
### **Xavier**
Layer의 특징에 따라서 초기화를 해야 한다!
n_in: layer input 수
n_out: layer ouput 수
- Nomal initalization   
$$W∼N(0, Var(W))$$
$$Var(W) = \sqrt{\frac{2}{n_{in} + n_{out}}}$$
- Uniform initialization
$$W∼U(-\sqrt{\frac{6}{n_{in} + n_{out}}}, +\sqrt{\frac{6}{n_{in} + n_{out}}})$$

### **He**
Xavier의 변형, n_out 안 쓰는 Xavier라 생각하면 됨.
- Nomal initalization   
$$W∼N(0, Var(W))$$
$$Var(W) = \sqrt{\frac{2}{n_{in}}}$$
- Uniform initialization
$$W∼U(-\sqrt{\frac{6}{n_{in}}}, +\sqrt{\frac{6}{n_{in}}})$$

## **Code: mnist_nn_xavier**

In [ ]:
def xavier_uniform_(tensor, gain=1): #gain = Var(W)
    fan_in, fan_out = calculate_fan_in_and_fan_out # n_in, n_out
    std = gain * math.sqrt(2.0 / (fan_in + fan_out))
    a= math.sqrt(3.0) * std #uniform initialization 형태
    with torch.no_grad():
        return tensor.uniform_(-a, a) #uniform distribution 초기화

In [ ]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#for reproducibility
random.seed(777) # 데이터 전처리 및 분할
torch.manual_seed(777) #랜덤 초기화
if device == 'cuda':
    torch.cuda.manual_seed_all

In [ ]:
# parameters
learning_rate = 0.01
training_epochs = 15
batch_size = 100

In [ ]:
#mnist dataset
mnist_train = dsets.MNIST(root='./',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)
mnist_test = dsets.MNIST(root='./',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [ ]:
#dataset loader
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                    batch_size = batch_size,
                                    shuffle = True,
                                    drop_last = True)

In [ ]:
#nn layers
linear1 = torch.nn.Linear(784, 512, bias=True)
linear2 = torch.nn.Linear(512, 512, bias=True)
linear3 = torch.nn.Linear(512, 512, bias=True)
linear4 = torch.nn.Linear(512, 512, bias=True)
linear5 = torch.nn.Linear(512, 10 , bias=True)
relu = torch.nn.ReLU()

#Xaviar initialization
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

Parameter containing:
tensor([[-0.0565,  0.0423, -0.0155,  ...,  0.1012,  0.0459, -0.0191],
        [ 0.0772,  0.0452, -0.0638,  ...,  0.0476, -0.0638,  0.0528],
        [ 0.0311, -0.1023, -0.0701,  ...,  0.0412, -0.1004,  0.0738],
        ...,
        [ 0.0334,  0.0187, -0.1021,  ...,  0.0280, -0.0583, -0.1018],
        [-0.0506, -0.0939, -0.0467,  ..., -0.0554, -0.0325,  0.0640],
        [-0.0183, -0.0123,  0.1025,  ..., -0.0214,  0.0220, -0.0741]],
       requires_grad=True)

In [ ]:
#model
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3, relu, linear4, relu, linear5).to(device)

In [ ]:
#optimizer, cost
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.374631554
Epoch: 0002 cost = 0.212149188
Epoch: 0003 cost = 0.182468370
Epoch: 0004 cost = 0.158700451
Epoch: 0005 cost = 0.145320401
Epoch: 0006 cost = 0.143682033
Epoch: 0007 cost = 0.151302010
Epoch: 0008 cost = 0.131958067
Epoch: 0009 cost = 0.117567852
Epoch: 0010 cost = 0.112223327
Epoch: 0011 cost = 0.108900353
Epoch: 0012 cost = 0.099287510
Epoch: 0013 cost = 0.103652336
Epoch: 0014 cost = 0.090350568
Epoch: 0015 cost = 0.103107490
Learning finished


In [ ]:
#Test the model
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test#dim=1
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    #Get one and predict
    r = random.randint(0, len(mnist_test)-1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())


/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9639999866485596
Label:  8
Prediction:  7
